In [1]:
from gurobipy import GRB
import gurobipy as gp
import numpy as np
np.random.seed(0)
import math

#### let's try to solve large scale cutting stock problem to see if the licence is adequately introduced to the environment.

In [2]:
class Instance(object):
    
    def __init__(self, num_order, min_order_len=10, max_order_len=50,
                min_demand=1, max_demand=5):
        
        # generates an instance which guarantees feasibility.
        self.n = num_order
        self.order_lens = np.random.randint(min_order_len, max_order_len+1, size=self.n)
        self.demands = np.random.randint(min_demand, max_demand+1, size=self.n)
        self.m = np.sum(self.demands)
        self.roll_len = max_order_len
    
    def summarize(self):
        print("Problem instance with ", self.n, " orders and ", self.m, "rolls")
        print("-"*47)
        print("\nOrders:\n")
        for i, order_len in enumerate(self.order_lens):
            print("\tOrder ", i, ": length= ", order_len, " demand=", self.demands[i])
        print("\nRoll Length: ", self.roll_len)

In [3]:
def optimize_kantorovich(ins_:Instance):
    
    # model
    model = gp.Model("kantorovich_formulation")
    
    # sets (ranges)
    rolls = range(ins_.m)
    orders = range(ins_.n)
    
    # decision variables
    use_roll = model.addVars(rolls,
                             vtype=GRB.BINARY,
                             obj=np.ones(ins_.m),
                             name="X")
    
    how_much_use = model.addVars(orders, rolls, 
                                 vtype=GRB.INTEGER,
                                 obj= np.zeros((ins_.n, ins_.m)), 
                                 name="Y")
    
    '''
    We could also use looping to 
    generate decision variables: 
    
    Example for "use_roll":
    -----------------------
    use_roll = []
    for j in rolls:
        use_roll.append(model.addVar(vtype=GRB.BINARY,
                                     obj=1,
                                     name="X[%d]" %p ))
    Example for "how_much_use":
    ---------------------------
    how_much_use = []
    for j in rolls:
        how_much_use.append([])
        for i in orders:
            how_much_use[j].append(model.addVar(...))
    '''
    
    # direction of optimization (min or max)
    model.modelSense = GRB.MINIMIZE
    
    # demand satisfaction constraint
    model.addConstrs(
                    (how_much_use.sum(i, '*') == ins_.demands[i] for i in orders), 
                    "Demand"
                    )
    
    # length constraint of a roll
    for j in rolls:
        model.addConstr(sum(how_much_use[i,j]*ins_.order_lens[i] for i in orders)
                        <= ins_.roll_len,
                        "Length[%d]" %j)
    
    # x-y link
    for i in orders:
        for j in rolls:
            model.addConstr(how_much_use[i,j] <= use_roll[j]*ins_.demands[i])
    
    if False:
        model.write('kantorovich.lp') # in case you want to write the model
    
    # solve
    model.optimize()
    
    # display objective function value
    print('\nTotal Number of Rolls Used: ', model.objVal)

In [4]:
instance = Instance(100)
instance.summarize()

Problem instance with  100  orders and  283 rolls
-----------------------------------------------

Orders:

	Order  0 : length=  10  demand= 1
	Order  1 : length=  13  demand= 2
	Order  2 : length=  13  demand= 1
	Order  3 : length=  49  demand= 3
	Order  4 : length=  19  demand= 3
	Order  5 : length=  29  demand= 4
	Order  6 : length=  31  demand= 3
	Order  7 : length=  46  demand= 4
	Order  8 : length=  33  demand= 3
	Order  9 : length=  16  demand= 2
	Order  10 : length=  34  demand= 3
	Order  11 : length=  34  demand= 4
	Order  12 : length=  22  demand= 4
	Order  13 : length=  11  demand= 4
	Order  14 : length=  48  demand= 3
	Order  15 : length=  49  demand= 4
	Order  16 : length=  33  demand= 5
	Order  17 : length=  34  demand= 2
	Order  18 : length=  27  demand= 3
	Order  19 : length=  47  demand= 4
	Order  20 : length=  35  demand= 2
	Order  21 : length=  23  demand= 3
	Order  22 : length=  18  demand= 2
	Order  23 : length=  19  demand= 5
	Order  24 : length=  30  demand= 3
	O

In [ ]:
optimize_kantorovich(instance)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-12
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 28683 rows, 28583 columns and 113200 nonzeros
Model fingerprint: 0x5ac7c015
Variable types: 0 continuous, 28583 integer (283 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 14971 rows and 0 columns
Presolve time: 0.26s
Presolved: 13712 rows, 28583 columns, 174356 nonzeros
Variable types: 0 continuous, 28583 integer (20093 binary)
Found heuristic solution: objective 188.0000000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Use crossover to convert LP symmetric solution to basic solution...
Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxatio